In [1]:
from predict import perform_inference
from rfw_loader import create_dataloaders
from train import train, write_model, save_model

import torch

In [2]:
DEVICE = 1
device = torch.device(f'cuda:{DEVICE}' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

In [3]:
EPOCHS = 5
LEARNING_RATE = 0.01
RATIO = 0.8
BATCH_SIZE = 32
DEVICE = 1

# Train ResNet18 for RFW

In [4]:
trainloader, testloader = create_dataloaders(
        "/media/global_data/fair_neural_compression_data/datasets/RFW/data_64", 
        "/media/global_data/fair_neural_compression_data/datasets/RFW/clean_metadata/clean.csv", 
        BATCH_SIZE, 
        RATIO
    )

In [10]:
next(iter(trainloader))[1].shape[1]

21

In [5]:
model = train(EPOCHS, LEARNING_RATE, trainloader, DEVICE)

Using Device  cuda:1


Using cache found in /home/rasta/.cache/torch/hub/pytorch_vision_v0.10.0
/home/rasta/.conda/envs/fnc_eval/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rasta/.conda/envs/fnc_eval/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,   500] loss: 0.477
[1,  1000] loss: 0.429
[2,   500] loss: 0.379
[2,  1000] loss: 0.365


KeyboardInterrupt: 